# 10分钟快速上手使用PaddleX——DeepLabV3+语义分割
> **当前教程文档基于PaddleX静态图版本（低于2.0.0)，如使用最新PaddleX动态图版本，请参考教程[PaddleX 2.0快速上手CV模型训练](https://aistudio.baidu.com/aistudio/projectdetail/2159977?channelType=0&channel=0)。**


***PaddleX简介***：PaddleX是飞桨全流程开发工具，集飞桨核心框架、模型库、工具及组件等深度学习开发所需全部能力于一身，打通深度学习开发全流程，并提供简明易懂的Python API，方便用户根据实际生产需求进行直接调用或二次开发，为开发者提供飞桨全流程开发的最佳实践。目前，该工具代码已开源于GitHub，同时可访问PaddleX在线使用文档，快速查阅读使用教程和API文档说明。  
***PaddleX代码GitHub链接***：[https://github.com/PaddlePaddle/PaddleX/tree/develop](https://github.com/PaddlePaddle/PaddleX/tree/develop)  
***PaddleX文档链接***：[https://paddlex.readthedocs.io/zh_CN/develop/index.html](https://paddlex.readthedocs.io/zh_CN/develop/index.html)   
***PaddleX官网链接***：[https://www.paddlepaddle.org.cn/paddle/paddlex](https://www.paddlepaddle.org.cn/paddle/paddlex)  
***PaddleX用户QQ群***: 1045148026  

### 前往GitHub给[PaddleX](https://github.com/PaddlePaddle/PaddleX)点击Star，关注项目，即可随时了解PaddleX的最新进展
![](https://ai-studio-static-online.cdn.bcebos.com/dc39ecb511664ca1a81bd3ad1d2bffa1c6e9dd1552934dedb119e14e6e8753dc)
  
## DeepLabV3+简介
DeepLabv3+在DeepLab模型的基础上，通过encoder-decoder进行多尺度信息的融合，提高了语义分割的健壮性和运行速率。本文档在一个小数据集上展示了如何通过PaddleX进行训练和预测。

## 1. 安装PaddleX

In [1]:
! pip install "paddlex<=2.0.0" -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 944 kB 6.6 MB/s            
     |████████████████████████████████| 1.5 MB 18.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.8 MB 18.8 MB/s            
     |████████████████████████████████| 288 kB 35.7 MB/s            
     |████████████████████████████████| 151 kB 67.5 MB/s            
     |████████████████████████████████| 106 kB 14.7 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 2.0 MB 16.2 MB/s            
     |████████████████████████████████| 50 kB 5.7 MB/s             
     |████████████████████████████████| 296 kB 25.5 MB/s            
     |████████████████████████████████| 98 kB 5.6 MB/s             
     |████████████████████████████████| 99 kB 7.0 MB/s             
  Preparing 

## 2. 准备视盘语义分割数据集

In [2]:
! wget https://bj.bcebos.com/paddlex/datasets/optic_disc_seg.tar.gz
! tar xzf optic_disc_seg.tar.gz

--2022-02-27 21:06:49--  https://bj.bcebos.com/paddlex/datasets/optic_disc_seg.tar.gz
Resolving bj.bcebos.com (bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19077558 (18M) [application/octet-stream]
Saving to: ‘optic_disc_seg.tar.gz’

optic_disc_seg.tar. 100%[===================>]  18.19M  12.5MB/s    in 1.5s    

2022-02-27 21:06:51 (12.5 MB/s) - ‘optic_disc_seg.tar.gz’ saved [19077558/19077558]



## 3. 模型训练

### 3.1 配置GPU

In [6]:
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

AttributeError: module 'paddle.distributed' has no attribute 'init_parallel_env'

### 3.2 定义图像预处理流程transforms
定义数据处理流程，其中训练和测试需分别定义，训练过程包括了部分测试过程中不需要的数据增强操作，如在本示例中，训练过程使用了`RandomHorizontalFlip`和`RandomPaddingCrop`两种数据增强方式，更多图像预处理流程transforms的使用可参见[paddlex.seg.transforms](https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/seg_transforms.html)。

In [7]:
from paddlex.seg import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(target_size=512),
    transforms.RandomPaddingCrop(crop_size=500),
    transforms.Normalize()
])
eval_transforms = transforms.Compose([
    transforms.Resize(512),
    transforms.Normalize()
])

AttributeError: module 'paddle.distributed' has no attribute 'init_parallel_env'

### 3.3 定义数据集Dataset
实例分割使用`SegDataset`格式的数据集，因此采用`pdx.datasets.SegDataset`来加载数据集，该接口的介绍可参见文档[pdx.datasets.SegDataset](https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html#paddlex-datasets-segdataset)。

In [5]:
train_dataset = pdx.datasets.SegDataset(
    data_dir='optic_disc_seg',
    file_list='optic_disc_seg/train_list.txt',
    label_list='optic_disc_seg/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.SegDataset(
    data_dir='optic_disc_seg',
    file_list='optic_disc_seg/val_list.txt',
    label_list='optic_disc_seg/labels.txt',
    transforms=eval_transforms)

NameError: name 'pdx' is not defined

### 3.4 模型开始训练
使用本数据集在P40上训练，如有GPU，模型的训练过程预估为13分钟左右；如无GPU，则预估为5小时左右。更多训练模型的参数可参见文档[paddlex.seg.DeepLabv3p](https://paddlex.readthedocs.io/zh_CN/develop/apis/models/semantic_segmentation.html#paddlex-seg-deeplabv3p)。模型训练过程每间隔save_interval_epochs轮会保存一次模型在save_dir目录下，同时在保存的过程中也会在验证数据集上计算相关指标，具体相关日志参见[文档](https://paddlex.readthedocs.io/zh_CN/develop/appendix/metrics.html#id9)。  

**AIStudio使用VisualDL查看训练过程中的指标变化**  
1. 点击左边菜单图标的『**可视化**』；  
2. 设置logdir，logdir的路径为训练代码中`save_dir`指定的目录下的vdl_log目录，例如`output/deeplab/vdl_log`  
3. 点击下方『**启动VisualDL服务按钮**』，再『**打开VisualDL**』即可

In [6]:
num_classes = len(train_dataset.labels)
model = pdx.seg.DeepLabv3p(num_classes=num_classes)
model.train(
    num_epochs=40,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.01,
    save_interval_epochs=1,
    save_dir='output/deeplab',
    use_vdl=True)

## 4 模型预测

使用模型进行预测，同时使用`pdx.seg.visualize`将结果可视化，可视化结果将保存到`./output/deeplab`下，其中`weight`代表原图的权重，即mask可视化结果与原图权重因子。
> 此处请在AIStudio Notebook页面的右上角菜单，选择重启执行器，以释放显存，重新加载训练好的模型

In [ ]:
import paddlex as pdx
model = pdx.load_model('output/deeplab/best_model')
image_name = 'optic_disc_seg/JPEGImages/H0005.jpg'
result = model.predict(image_name)
pdx.seg.visualize(image_name, result, weight=0.4, save_dir='./output/deeplab')

可视化结果如下所示：  
<img src="https://ai-studio-static-online.cdn.bcebos.com/7f2cc3dd92c945da9038e68aa5bebab6bce58083b3fb415c8efd7d8257fcc2d6" width="50%" height="50%">       

## 5. 反馈
PaddleX用户QQ群: 1045148026 (手机QQ扫描如下二维码快速加入)  
<img src="https://ai-studio-static-online.cdn.bcebos.com/35a3a4c0748f4eb393b3acb100bc14f2e9920eed74244c1bbc2041bdd6790f81" width="15%" height="15%">
